## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [14]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad): 
        self.codigo = codigo
        self.nombre = nombre
        self._precio_ = precio  
        self.tipo = tipo
        self._cantidad_ = cantidad

    @property
    def precio(self): 
        return self._precio_

    @precio.setter
    def precio(self, valor):  
        if valor >= 0:
            self._precio_ = valor            
            
    @property
    def cantidad(self): 
        return self._cantidad_

    @cantidad.setter
    def cantidad(self, valor):  
        if valor >= 0:
            self._cantidad_ = valor
            
    def valorTotal(self):
        return self._precio_ * self._cantidad_

In [15]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [17]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos
        
    def esAplicable(self, producto):
        codigo_existe = producto.codigo in self.codigos
        tipo_existe = producto.tipo in self.tipos
        
        return codigo_existe or tipo_existe
        
    def aplicar(self, producto, cantidad):
        return cantidad * producto.precio  

        
        

class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        super().__init__(descripcion="Descuento 10%", *args, **kwargs)
        self.descuento = descuento  
    
    def aplicar(self, producto, cantidad):
        if self.esAplicable(producto):
            precio_total = cantidad * producto.precio
            descuento_aplicado = precio_total * (self.descuento / 100)
            return precio_total - descuento_aplicado
        return cantidad * producto.precio  

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            precio_total = cantidad * producto.precio
            return precio_total * (self.descuento / 100)
        return 0

        
    
class Oferta2x1(Oferta):
    def __init__(self, *args, **kwargs):
        super().__init__(descripcion="Oferta 2x1", *args, **kwargs)
        
    def aplicar(self, producto, cantidad):
        if self.esAplicable(producto):
            cantidad_a_pagar = (cantidad // 2) + (cantidad % 2)
            return cantidad_a_pagar * producto.precio
        return cantidad * producto.precio  

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            if cantidad < 2:  
                return 0
            else:
                precio_con_promo = (cantidad // 2) * producto.precio
                return precio_con_promo
        return 0

In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [18]:
import csv

class Catalogo:
    def __init__(self):
        self.Productos = []
        self.cantidadProductos = len(self.Productos)
        self.cantidadUnidades = 0
        self.valorTotal = 0
        self.ofertas_registradas = []
        
        
    def agregar(self, *productos):
        for producto in productos:  
            self.Productos.append(producto)  
            self.cantidadProductos = len(self.Productos)
            self.cantidadUnidades += producto.cantidad
            self.valorTotal += producto.cantidad * producto.precio

       
    def registrarOferta(self, oferta):
        self.ofertas_registradas.append(oferta)
        
    def calcularDescuento(self, Producto, cantidad):
        for of in self.ofertas_registradas:
            if of.esAplicable(Producto):
                return of.calcularDescuento(Producto, cantidad)  
        else:
            return 0  
        
        
    def guardar(self, ruta):
        with open(ruta, "w", newline="") as c:
            escribir_nuevo_producto = csv.writer(c, delimiter=",")
            escribir_nuevo_producto.writerow(["codigo", "nombre", "precio", "tipo", "cantidad"])

            for producto in self.Productos:
                escribir_nuevo_producto.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])
    
    
    def vender(self,Producto, cantidad):   
        if cantidad > Producto.cantidad:
            return "La cantidad a vender es mas que el stock del producto"
        else:
          Producto.cantidad -= cantidad  
          self.cantidadUnidades -= cantidad
          self.valorTotal -= cantidad * Producto.precio
            

    
    def informe(self):
        
        precio_total = sum( p.precio for p in self.Productos)
        precio_promedio = precio_total / self.cantidadProductos
        
        cantidad_gaseosa = 0
        total_precio_gaseosa = 0
        cantidad_galleta = 0
        total_precio_galleta = 0

        for producto in self.Productos:
            if producto.tipo == "gaseosa":
                cantidad_gaseosa += producto.cantidad 
                total_precio_gaseosa += producto.precio * producto.cantidad  
                
            elif producto.tipo == "galleta":
                cantidad_galleta += producto.cantidad  
                total_precio_galleta += producto.precio * producto.cantidad 

    
        precio_promedio_gaseosa = total_precio_gaseosa / cantidad_gaseosa 
        precio_promedio_galleta = total_precio_galleta / cantidad_galleta 
        
        
        print("\n")
        print("INFORME CATALOGO")
        informe_estructura = f"""Cantidad de productos:             {self.cantidadProductos}
Cantidad de unidades:             {self.cantidadUnidades}
Precio Promedio:                {precio_promedio}
Valor total:                  {self.valorTotal}
Tipos de productos: 
- gaseosa :                   unidades: {cantidad_gaseosa} x precio promedio: {precio_promedio_gaseosa}
- galleta :                   unidades: {cantidad_galleta} x precio promedio: {precio_promedio_galleta:.2f}

Ofertas:
        """
        
        for oferta in self.ofertas_registradas: 
            informe_estructura += f"- {oferta.descripcion}\n"
       
        print(informe_estructura)
        return informe_estructura
        

    def buscar(self, codigo):
            for p in self.Productos:
                if p.codigo == codigo: 
                    return p  
            return None
        
        
    @classmethod
    def leer(cls, ruta):
        catalogo = cls()
 
        with open(ruta, "r", newline="") as c:
            lector = csv.reader(c)
            next(lector)  
        
            for fila in lector:
                producto = Producto(fila[0], fila[1], float(fila[2]), fila[3], int(fila[4]))
                catalogo.Productos.append(producto)

        catalogo.cantidadProductos = len(catalogo.Productos)
        catalogo.cantidadUnidades = sum(p.cantidad for p in catalogo.Productos)
        catalogo.valorTotal = sum(p.cantidad * p.precio for p in catalogo.Productos)

        return catalogo 

In [19]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")



INFORME CATALOGO
Cantidad de productos:             4
Cantidad de unidades:             97
Precio Promedio:                1550.0
Valor total:                  163400
Tipos de productos: 
- gaseosa :                   unidades: 30 x precio promedio: 1300.0
- galleta :                   unidades: 67 x precio promedio: 1856.72

Ofertas:
        - Oferta 2x1
- Descuento 10%

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [20]:
class Cliente: 
    def __init__(self, nombre, cuit):
        self._nombre_= nombre
        self._cuit_ = cuit
        
    @property
    def nombre(self):
        return self._nombre_
        
    @nombre.setter
    def nombre(self, valor): 
        if valor != "": 
            self._nombre_ = valor
            return  self._nombre_
    
    @property
    def cuit(self):
        return self._cuit_
        
    @cuit.setter
    def cuit(self, valor): 
        ante_guion = valor[0:2]
        entre_guion = valor[3:11]
        despues_guion = valor[0:2]
        
        if len(valor) == 13:
            if valor[2] == "-" and valor[11] == "-":
                if ante_guion.isnumeric() and entre_guion.isnumeric() and despues_guion.isnumeric():
                    self._cuit_ = valor
                    return self._cuit_

In [21]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [22]:
from datetime import datetime

class Factura:
    numero_factura_secuencial = 0  
    
    def __init__(self,catalogo, cliente):
        self.cliente = cliente
        self.catalogo = catalogo
        self.numero = Factura.nuevoNumero()
        self.productos_en_factura = []
        self.cantidadProductos = 0
        self.cantidadUnidades = 0
        self.subtotal = 0
        self.total = 0
        self.fecha = datetime.today().date()
        
        
    @classmethod
    def ultimaFactura(cls, numero):
        cls.numero_factura_secuencial = numero 

    @classmethod
    def nuevoNumero(cls):
        cls.numero_factura_secuencial += 1  
        return cls.numero_factura_secuencial    
    
    
    def aplicar_descuentos(self):
        self.descuentos = sum(
            self.catalogo.calcularDescuento(p, p.cantidad) 
            for p in self.productos_en_factura
        )
            
    
    def agregar(self, producto, cantidad):
        for pf in self.productos_en_factura:
            if pf.nombre == producto.nombre:
                pf.cantidad += cantidad 
                break
                     
        else:
            producto.cantidad = cantidad  
            self.productos_en_factura.append(producto)  
        
        self.cantidadProductos = len(self.productos_en_factura)
        self.cantidadUnidades = sum(pf.cantidad for pf in self.productos_en_factura)
        self.subtotal = sum(pf.precio * pf.cantidad for pf in self.productos_en_factura)
        self.aplicar_descuentos()
        self.total = self.subtotal - self.descuentos
        
        self.catalogo.cantidadUnidades -= cantidad
        
    
    def imprimir(self):
        producto1 = self.productos_en_factura[0]  
        descuento1 = self.catalogo.calcularDescuento(producto1, producto1.cantidad)
        subtotal1 = producto1.precio * producto1.cantidad

        producto2 = self.productos_en_factura[1] 
        descuento2 = self.catalogo.calcularDescuento(producto2, producto2.cantidad)
        subtotal2 = producto2.precio * producto2.cantidad

        productos_str = (
        f"- {producto1.cantidad} {producto1.nombre:<38} x ${producto1.precio:.2f} = ${subtotal1:.2f}\n"
        f"{'':>44} Descuento: - ${descuento1:.2f}\n"
        f"{'':>44} Oferta: Descuento 10%\n"
        f"- {producto2.cantidad} {producto2.nombre:<38} x ${producto2.precio:.2f} = ${subtotal2:.2f}\n"
        f"{'':>44} Descuento: - ${descuento2:.2f}\n"
        f"{'':>44} Oferta: Oferta 2x1\n"
    )
    
        factura = (
        f"Factura: {self.numero}\n"
        f"Fecha  : {self.fecha}\n"
        f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"
        f"{productos_str}\n"
        f"{'':>45} Subtotal:   ${self.subtotal:.2f}\n"
        f"{'':>45} Descuentos: ${self.descuentos:.2f}\n"
        f"{'':>45} -----------------------\n"
        f"{'':>45} TOTAL:      ${self.total:.2f}\n"
    )

        print(factura)
        return factura

In [23]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Factura: 103
Fecha  : 2024-10-05
Cliente: Juan Perez (20-12345678-9)

- 10 Coca Cola                              x $1500.00 = $15000.00
                                             Descuento: - $1500.00
                                             Oferta: Descuento 10%
- 3 Sonrisa                                x $1200.00 = $3600.00
                                             Descuento: - $1200.00
                                             Oferta: Oferta 2x1

                                              Subtotal:   $18600.00
                                              Descuentos: $2700.00
                                              -----------------------
                                              TOTAL:      $15900.00

Prueba pasada exitosamente!


In [24]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
